In [1]:
# Path setup to resolve package imports (dynamic project root)
import sys, os

# Derive project root from current notebook directory: <project>/notebook
notebook_dir = os.getcwd()
project_root = os.path.dirname(notebook_dir)

if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [6]:
from src.agents.tools.db_tools import DBToolManager
# Test tool manager
tool_mgr = DBToolManager()
tools = tool_mgr.get_tools()
print(f"✅ {len(tools)} tools ready")
print("Tool names:", [t.name for t in tools])


✅ 4 tools ready
Tool names: ['list_all_tables', 'get_table_schema', 'preview_sql', 'execute_sql']


In [7]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from src.agents.prompt import system_prompt

checkpointer = MemorySaver()

gptLLM = ChatOpenAI(
    model_name="gpt-4",temperature=0
)

agent = create_agent(
    model=gptLLM,
    tools = tools,  # Add your tools here when ready
    system_prompt=system_prompt,
    checkpointer=checkpointer  # Use your custom ReAct prompt
)

config = {"configurable": {"thread_id": "user_123"}}


In [8]:
question = "employees in Sales department HumanResources"
result = agent.invoke({
    "messages": [{"role": "user", "content": question}]
}, config=config)

print(f"\nAgent: {result['messages'][-1].content}")


Agent: The query was executed successfully and returned the following results:

1. Employee with `BusinessEntityID` 15, Job Title 'Design Engineer', hired on 2011-01-18, in Department 1.
2. Employee with `BusinessEntityID` 14, Job Title 'Senior Design Engineer', hired on 2010-12-30, in Department 1.
3. Employee with `BusinessEntityID` 2, Job Title 'Vice President of Engineering', hired on 2008-01-31, in Department 1.
4. Employee with `BusinessEntityID` 6, Job Title 'Design Engineer', hired on 2008-01-24, in Department 1.
5. Employee with `BusinessEntityID` 5, Job Title 'Design Engineer', hired on 2008-01-06, in Department 1.

These are the top 5 most recently hired employees in the Sales department (assuming Department 1 is the Sales department).
